In [29]:
import os
import copy
import sys
sys.path.append('D:\Repo\sensoreval')
from filedriver import cmpfile
import json 

In [30]:
# 获得cmp文件的数据总帧数

def get_FrameNum(WorkFile):
    
    FIndx = cmpfile.DataTypeId.FrameIndex
    FEnd = copy.deepcopy(WorkFile.ReadFrame(0,FIndx))
    FrameNum = FEnd.DataBlock.FrameIdxNum
    
    return FrameNum

In [31]:
def find_added_stillin_removed(current,last):
    added = []
    stillin = []
    removed = []
    for i in current:
        if i in last:
            stillin.append(i)
        else:
            added.append(i)
    for i in last:
        if i not in current:
            removed.append(i)
    return added,stillin,removed

In [40]:
def cmp_classification_extract(fPath):
    
    WorkFile = cmpfile.CmpFile(fPath)

    #输出数据帧总数
    FrameNum = get_FrameNum(WorkFile)
    
    FrameInfo = {}
    huanchong = {}
    
    dt = cmpfile.DataTypeId.Object
    
    for i in range(FrameNum):
        currentF = copy.deepcopy(WorkFile.ReadFrame(i,dt,'begin'))
        for _ in currentF.DataBlock.Objects:
            huanchong[_.Id]= _.Classification.name
            
        FrameInfo[i] = huanchong
        huanchong = {}
    
    huanchong = {}
    tmp = {}
    last_ids = []
    rst = []

    for _ in range(FrameNum):
        
        current_info = FrameInfo[_]
        current_ids = current_info.keys()
        added,stillin,removed = find_added_stillin_removed(current_ids,last_ids)

        for i in added:
            huanchong[i] = [current_info[i]]
        for i in stillin:
            if current_info[i] not in huanchong[i]:
                huanchong[i].append(current_info[i])
    
        for i in removed:
            tmp = {i:huanchong[i]}
            rst.append(tmp)
            del huanchong[i]

        if _ == FrameNum - 1:
            for key, value in huanchong.items():
                rst.append({key:value})

        last_ids = current_ids
    
    # 写入 Json 文件
    
    fname = 'DataOrigin_process_2.json'
    with open(fname, 'w') as f:
        json.dump(rst,f,indent = 4)
    
    print("文件中物体分类读取已完成，保存到了Json文件中,Json 文件的保存地址为：")
    print(os.path.abspath(fname))

In [41]:
fPath = r"D:\jiaojie\to_csv\bike_007.cmp"

cmp_classification_extract(fPath)

文件中物体分类读取已完成，保存到了Json文件中,Json 文件的保存地址为：
D:\jiaojie\Relabel\DataOrigin_process_2.json
